In [461]:
import os
import time
import openai

In [462]:
dataDirectory = "Dataset/ijcnlp_dailydialog"
textFile = dataDirectory + "/dialogues_text.txt"
emotionFile = dataDirectory + "/dialogues_emotion.txt"


In [463]:
def readLinebyLine(filePath,maxLine=float('inf')):
    currLine=0
    with open(filePath, 'r') as file:
    # Loop through each line in the file
        for line in file:
            if(currLine==maxLine):
                return
            # Process each line here
            print(line.strip())  # Print the line after stripping whitespace (e.g., newline characters)
            currLine+=1

In [464]:
def extractEmotionConversations(emotionFile, textFile, label):
    conversationNumbers = set()
    lineNumber = 0
    count=0
    with open(emotionFile, 'r') as file:
    # Loop through each line in the file
        for line in file:

            if(label in line):
                conversationNumbers.add(lineNumber)
                count+=line.count(label)
            lineNumber+=1
    #print(len(conversationNumbers), count)
    return conversationNumbers

In [571]:
def attachLabels(line):
    rawConvo=line.split('__eou__')
    proConvo = [item for item in rawConvo if item != '\n']
    for i in range(len(proConvo)):
        if (i%2 == 1):
            proConvo[i] = "Person 2: " + proConvo[i]
        else:
            proConvo[i] = "Person 1: " + proConvo[i]
    return proConvo

def prepareString(withLabels,currentIndex,history):
    currString = ''
    if(history == 0):
        return withLabels[currentIndex]
    elif(history ==1):
        if(currentIndex==0):
            return withLabels[0]
        else:
            currString += withLabels[currentIndex-1] + '\n'
            currString += withLabels[currentIndex] + '\n'
            return currString
    elif history == 2:
        for i in range(0,currentIndex+1):
            currString = currString + withLabels[i] + "'\n"
        return currString
    else:
        raise ValueError("Invalid History:", history)

def preparePrompt(stringGPT, context,currLine,message):
    finalPrompt = stringGPT
    if(context):
        finalPrompt = "The topic of the following conversation is " + getContext(currLine)+".\n" + finalPrompt
    else:
        finalPrompt = "Here is a conversation snippet:" +"\n" + finalPrompt

    if ('@rukhshan@' in finalPrompt):
        finalPrompt = finalPrompt.replace("@rukhshan@",message)
        #print("Final Prompt", finalPrompt)
    return finalPrompt

def callGPT(finalPrompt):
    #openai.api_type = "azure"
    #print("Prompt:\n", finalPrompt,'\n\n')
    openai.api_base = "REMOVED"
    openai.api_key = 'REMOVED'
    openai.api_version = "2023-05-15"
    response = openai.ChatCompletion.create(
        engine="GPT4",
        messages=[

            {"role": "user", "content": finalPrompt}
        ]
    )

    if ("N" in response['choices'][0]['message']['content']):
        return '0'
    elif("Y" in response['choices'][0]['message']['content']):
        return '1'
    elif("M" in response['choices'][0]['message']['content']):
        return '2'
    else:
        print("K Produced",response['choices'][0]['message']['content'], "\nPrompt:", finalPrompt)
        return 'K'
    
def writeOutput(outputGPT, promptChosen,writeFilePath):

    finalOutput = promptChosen.replace("\n","") + "\n" + outputGPT

    # Open the file in write mode ('w')
    # This will create the file if it doesn't exist or completely overwrite it if it does
    with open(writeFilePath, 'w') as file:
        file.write(finalOutput)

    # The file is automatically closed when the 'with' block exits

def getContext(currLineIndex):
    with open("/Users/rukhshanharoon/Desktop/LLMChattingApp/Analysis/Dataset/ijcnlp_dailydialog/dialogues_topic.txt", "r") as file:
        # Read all lines into a list
        lines = file.readlines()
    contextNumber = lines[currLineIndex].replace("\n","")
    if(contextNumber=="1"):
        return "Ordinary Life"
    elif(contextNumber=="2"):
        return "School Life"
    elif(contextNumber=="3"):
        return "Culture & Education"
    elif(contextNumber=="4"):
        return "Attitude & Emotion"
    elif(contextNumber=="5"):
        return "Relationship"
    elif(contextNumber=="6"):
        return "Tourism"
    elif(contextNumber=="7"):
        return "Health"
    elif(contextNumber=="8"):
        return "Work"
    elif(contextNumber=="9"):
        return "Politics"
    elif(contextNumber=="10"):
        return "Finance"
    else:
        raise ValueError("Invalid contextNumber:", contextNumber)
    
def readSpecificConversations(filePath,conversationNumbers,promptChosen,writeFilePath, history=2,context=False, max=float('inf'), startFrom=0):
    currLine=0
    currOutput=''
    currCount=0
    reqCount=0
    with open(filePath, 'r') as file:
    # Loop through each line in the file
        for line in file:
            if(currLine in conversationNumbers and currLine>=startFrom):
                convoPrinted = False
                withLabels = attachLabels(line)
                for i in range(len(withLabels)):
                    convoGPT = prepareString(withLabels,i,history)
                    finalPrompt = preparePrompt(convoGPT + promptChosen,context,currLine, withLabels[i])
                    outputGPT=""
                    GPTdone = False
                    while(GPTdone==False):
                        try:  
                            outputGPT = callGPT(finalPrompt)
                            reqCount+=1
                            GPTdone=True
                        except Exception as e:
                            #prints index
                            #line number is index + 1
                            #print("Conversation index: ", currLine)
                            if('response was filtered due to the prompt triggering' in str(e)):
                                print("Skipped by GPT.\n" )    
                                if(convoPrinted==False):
                                    print("Convo:\n", convoGPT)
                                    convoPrinted = True
                                outputGPT = "S"
                                GPTdone=True
                            else:
                                #print("Sleeping for 8.")
                                time.sleep(8)
                    currOutput = currOutput +" " + outputGPT
                #currOutput = currOutput + "\n"
                currCount+=1
                if(currCount==max):
                    writeOutput(currOutput, promptChosen,writeFilePath)
                    print("Total Requests:", reqCount)
                    return '\n'.join([line.lstrip() for line in currOutput.splitlines()])
                else:
                    currOutput = currOutput + "\n"
            
            currLine+=1
    writeOutput(currOutput, promptChosen,writeFilePath)
    print("Total Requests:", reqCount)
    return '\n'.join([line.lstrip() for line in currOutput.splitlines()])


In [459]:
def extractLabels(emotionFile, label, max=float('inf')):
    conversations =""
    lineNumber = 0

    with open(emotionFile, 'r') as file:
    # Loop through each line in the file
        for line in file:
            if(lineNumber == max):
                return conversations

            if(label in line):
                newLine=""
                for i in range (len(line)):
                    if((line[i] != " ") and (line[i] != "\n") and (line[i] != "1")):
                        newLine += '0'
                    else:
                        newLine +=line[i]
                conversations += newLine
                lineNumber+=1

        
    #print(len(conversationNumbers), count)
    return conversations

In [309]:
def writeRealLabels(labels, name):
    with open(name, 'w') as file:
        file.write(labels)

In [569]:
def processData(realLabels, genLabelsName):

    processedRealLabels=""
    processedGenLabels=""
    index=[]
    selectedIndex=[]
    indexCount=0
    n=0
    
    
    firstLine = False
    with open(genLabelsName, 'r') as file:
        for line in file:
            # Skip first line
            if(firstLine==False):
                firstLine=True
            elif 'S' not in line or 'K' not in line:
                # If it doesn't contain "S", add the line to the processed content
                cleanedLine = line.replace("\n", "").replace(" ", "")
                processedGenLabels += cleanedLine
                selectedIndex.append(indexCount)
                indexCount+=1
                n+=1
                
            else:
                #indexes of lines that contain S in REAL file
                index.append(indexCount)
                indexCount+=1
    
    print(selectedIndex)
    indexCount=0
    lineCount=0
    with open(realLabels, 'r') as file:
        for line in file:
            lineCount+=1
            if ((indexCount in index) or not(indexCount in selectedIndex)):
                indexCount+=1
            else:
                indexCount+=1
                cleanedLine = line.replace("\n", "").replace(" ", "")
                for char in cleanedLine:
                    if(char =='1'):
                        processedRealLabels += char
                    else:
                        processedRealLabels += '0'            
    print(processedRealLabels)

    return processedRealLabels, processedGenLabels,n
    

In [501]:
#Interpretng Precision
#Precision is a measure of true positives / true positives + false positives
#Shows how many positives are true from the set of all elements marked as positive by the model
#TP/(TP+FP)
#5/(5+5) -> 0.5 means that out of all elements it marked as positive, only half were actually positive and the rest were negative. the higher the value, higher the precision
#higher the precision, higher the quality of positive predictions

#Interpreting Recall
#shows the proportion of all true positives that the model was able to classify as positive 
#TP/(TP + FN)
#5/(5+5) -> shows that it was only able to detect 5 out of the 10 true positive elements.
#higher the recall, higher the sensitivity
def PRFA(angryOnlyRealLabels, angryOnlyGenLabels,n, promptNumber, ITConvos, Prompt, Context):
    # Initialize counters for True Positives (TP), False Positives (FP), True Negatives (TN), False Negatives (FN)
    TP = FP = TN = FN = 0
    '''
    # Calculate TP, FP, TN, FN
    for true_label, predicted_label in zip(angryOnlyRealLabels, angryOnlyGenLabels):
        if (true_label == '1' and (predicted_label == '1' or predicted_label == "2")):
            TP += 1
        elif true_label == '0' and predicted_label == '1':
            FP += 1
        elif true_label == '0' and (predicted_label == '0' or predicted_label== "2"):
            TN += 1
        elif true_label == '1' and predicted_label == '0':
            FN += 1
    '''
    for true_label, predicted_label in zip(angryOnlyRealLabels, angryOnlyGenLabels):
        if (true_label == '1' and predicted_label == '1'):
            TP += 1
        elif true_label == '0' and predicted_label == '1':
            FP += 1
        elif true_label == '0' and predicted_label == '0':
            TN += 1
        elif true_label == '1' and predicted_label == '0':
            FN += 1
    
    # Calculate Precision, Recall, F1 Score, and Accuracy
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)
    accuracy = (TP + TN) / (TP + TN + FP + FN)

    dataset = {
    "#Prompt":,
    "#ITConvos":,
    "Prompt":,
    "Context":,
    "Precision": precision,
    "Recall": recall,
    "F1": f1_score,
    "Accuracy": accuracy,
    "FP": FP,
    "FN": FN,
    "TP": TP,
    "TN": TN,
    "#FTMessages": FP + FN + TP + TN,
    "#FTConvos": n1
}

    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1_score)
    print("Accuracy:", accuracy)
    print("FP", FP)
    print("FN", FN)
    print("TP",TP)
    print("TN",TN)
    print("Total:", FP+FN+TP+TN)
    print("Total Convos:",n)



In [466]:
#conversations with at least one message labeled as angry
angryConversations = extractEmotionConversations(emotionFile,textFile,'1')
print("Conversation indexes with at least 1 angry emotion:\n",angryConversations)

Conversation indexes with at least 1 angry emotion:
 {1, 2, 10251, 12301, 15, 32, 2084, 4132, 4134, 10281, 4143, 12337, 2100, 2101, 2102, 2106, 2111, 2113, 4161, 4165, 6214, 76, 2126, 4179, 4182, 8278, 8283, 2140, 4188, 95, 96, 97, 2148, 4196, 12395, 4205, 4208, 4211, 12403, 2168, 4216, 4217, 12408, 4220, 8318, 8321, 4227, 4228, 8324, 4231, 136, 12425, 4237, 4243, 10388, 10389, 10390, 10392, 10393, 155, 4251, 157, 10395, 10396, 4256, 4257, 4259, 4263, 6139, 2218, 10413, 12653, 2223, 2224, 4273, 178, 10415, 2228, 10416, 2230, 4278, 4280, 186, 4283, 2236, 2237, 10429, 2239, 2241, 2243, 4308, 2262, 4310, 4314, 222, 4320, 4321, 2275, 4327, 2280, 8428, 2285, 12530, 4348, 2302, 4354, 8452, 4362, 2324, 10517, 4377, 4379, 4387, 295, 296, 6440, 8487, 300, 4398, 306, 311, 8503, 4414, 10564, 2374, 327, 4423, 8521, 4426, 4427, 2380, 8522, 8526, 2383, 2384, 4431, 338, 2387, 8534, 2391, 351, 2402, 4451, 12642, 357, 364, 2413, 4460, 2415, 368, 2416, 370, 4465, 12284, 12652, 4470, 12656, 4472, 2427, 3

In [467]:
labels = extractLabels(emotionFile, '1', 40)
writeRealLabels(labels, "angryReal40.txt")

In [449]:
prompts = [
    '\nState if the tone/intent of the last message in the conversation above is negative or not this format: "Y" for yes and "N" for no.',
    '\nState if the tone/intent of the last message in the conversation above is negative (even if it is slightly negative) or not. Format output like this: "Y" for yes and "N" for no.',
    '\nState if the tone/intent of the last message in the conversation above is negative (only those which are extremely negative) or not. Format output like this: "Y" for yes and "N" for no.',
    '\nState if the tone/intent of the last message in the conversation above is extremely negative, clearly not negative, or maybe negative. Format output like this: "Y" for Extremely Negative and "N" for Clearly Not Negative and "M" for Maybe Negative.',
    '\nWould the last message in the conversation above be perceived as blunt/negative/offensive by its recipient? Format your output like this: "Y" if Yes and "N" if No and "M" if Maybe.',
    '\nWould the last message in the conversation above be perceived as blunt/negative/offensive by its recipient? Format your output like this: "Y" if Yes and "N" if No.'
]

**Prompt 10**

In [511]:
prompt = 'State if the tone/intent of the last message in the conversation above is negative (even if it is slightly negative) or not. Format output like this: "Y" for yes and "N" for no.'

History: 0
Context: Yes

In [ ]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output10/output-10-1-Y.txt",history =1, context=True, max=40)

In [509]:
#reads output file
#records labels AND indexes for all convos that were NOT skipped
#reads angryReal40.txt
#extracts labels for all convos that were NOT skipped by matching their indexes with the indexes collected previously
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output10/output-10-0-Y.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 264 264


**Stats**


Precision: 0.44871794871794873
Recall: 0.7291666666666666
F1 Score: 0.5555555555555556
Accuracy: 0.7878787878787878
FP 43
FN 13
TP 35
TN 173
Total: 264
Total Convos: 37


History: 1
Context: Y

In [512]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output10/output-10-1-Y.txt",history =1, context=True, max=40)

Skipped by GPT.

Convo:
 Person 1:  And after that ? 
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 

Skipped by GPT.

Convo:
 Person 1:  What ? He can't do this to you . 
Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 

Skipped by GPT.

Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Person 2:  We'll see what happens after court . 

Total Requests: 294


In [513]:
#reads output file
#records labels AND indexes for all convos that were NOT skipped
#reads angryReal40.txt
#extracts labels for all convos that were NOT skipped by matching their indexes with the indexes collected previously
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output10/output-10-1-Y.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 264 264


**Stats**


Precision: 0.47435897435897434
Recall: 0.7708333333333334
F1 Score: 0.5873015873015872
Accuracy: 0.803030303030303
FP 41
FN 11
TP 37
TN 175
Total: 264
Total Convos: 37


History: 2
Context: Y

In [516]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output10/output-10-2-Y.txt",history =2, context=True, max=40)

Skipped by GPT.

Convo:
 Person 1: Mom , I want to get a motorcycle . Is that all right with you ? '
Person 2:  You mean a murder cycle ? Did you know more cyclists die in accidents than motorists ? '
Person 1:  Mom ! I'll wear a helmet and I'll drive sane . I promise . '
Person 2:  I'll tell you what . You can get a motorcycle on one condition . '
Person 1:  What's that ? '
Person 2:  You spend two weeks with dad in his ambulance as an EMT trainee and volunteer at the emergency room for one month . '
Person 1:  And after that ? '
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . '

Skipped by GPT.

Convo:
 Person 1: What's up ? You sound a little down in dumps . '
Person 2:  I quarreled with my roommate last night . He is really stubborn . '
Person 1:  Calm down ! Shouting won ’ t help ? '
Person 2:  He is really outrageous . '
Person 1:  What happened ? '
Person 2:  I went back home last night . You know tired as a dog , so I took a quick s

In [518]:
#reads output file
#records labels AND indexes for all convos that were NOT skipped
#reads angryReal40.txt
#extracts labels for all convos that were NOT skipped by matching their indexes with the indexes collected previously
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output10/output-10-2-Y.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 274 274


**Stats**


Precision: 0.42857142857142855
Recall: 0.78
F1 Score: 0.5531914893617021
Accuracy: 0.7700729927007299
FP 52
FN 11
TP 39
TN 172
Total: 274
Total Convos: 38


History: 0
Context: N

In [522]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output10/output-10-0-N.txt",history =0, context=False, max=40)

Skipped by GPT.

Convo:
 Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 
Skipped by GPT.

Convo:
 Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 
Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Total Requests: 295


In [523]:
#reads output file
#records labels AND indexes for all convos that were NOT skipped
#reads angryReal40.txt
#extracts labels for all convos that were NOT skipped by matching their indexes with the indexes collected previously
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output10/output-10-0-N.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 264 264


**Stats**


Precision: 0.4375
Recall: 0.7291666666666666
F1 Score: 0.546875
Accuracy: 0.7803030303030303
FP 45
FN 13
TP 35
TN 171
Total: 264
Total Convos: 37


History: 1
Context: N

In [524]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output10/output-10-1-N.txt",history =1, context=False, max=40)

Skipped by GPT.

Convo:
 Person 2:  What ? 
Person 1:  Any body piercing ? Stick out your tongue ! 

Skipped by GPT.

Convo:
 Person 1:  And after that ? 
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 

Skipped by GPT.

Convo:
 Person 1:  What ? He can't do this to you . 
Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 

Skipped by GPT.

Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Person 2:  We'll see what happens after court . 

Total Requests: 293


In [525]:
#reads output file
#records labels AND indexes for all convos that were NOT skipped
#reads angryReal40.txt
#extracts labels for all convos that were NOT skipped by matching their indexes with the indexes collected previously
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output10/output-10-1-N.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 256 256


**Stats**


Precision: 0.46153846153846156
Recall: 0.7659574468085106
F1 Score: 0.5760000000000001
Accuracy: 0.79296875
FP 42
FN 11
TP 36
TN 167
Total: 256
Total Convos: 36


History: 2 Context: N

In [526]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output10/output-10-2-N.txt",history =2, context=False, max=40)

Skipped by GPT.

Convo:
 Person 1: Mom , I want to get a motorcycle . Is that all right with you ? '
Person 2:  You mean a murder cycle ? Did you know more cyclists die in accidents than motorists ? '
Person 1:  Mom ! I'll wear a helmet and I'll drive sane . I promise . '
Person 2:  I'll tell you what . You can get a motorcycle on one condition . '
Person 1:  What's that ? '
Person 2:  You spend two weeks with dad in his ambulance as an EMT trainee and volunteer at the emergency room for one month . '
Person 1:  And after that ? '
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . '

Skipped by GPT.

Convo:
 Person 1: What's up ? You sound a little down in dumps . '
Person 2:  I quarreled with my roommate last night . He is really stubborn . '
Person 1:  Calm down ! Shouting won ’ t help ? '
Person 2:  He is really outrageous . '
Person 1:  What happened ? '
Person 2:  I went back home last night . You know tired as a dog , so I took a quick s

In [527]:
#reads output file
#records labels AND indexes for all convos that were NOT skipped
#reads angryReal40.txt
#extracts labels for all convos that were NOT skipped by matching their indexes with the indexes collected previously
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output10/output-10-2-N.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 259 259


**Stats**


Precision: 0.4523809523809524
Recall: 0.7755102040816326
F1 Score: 0.5714285714285715
Accuracy: 0.7799227799227799
FP 46
FN 11
TP 38
TN 164
Total: 259
Total Convos: 37


Prompt 11

In [528]:
prompt = 'State if the tone/intent of the last message in the conversation above is negative (only if it is extremely negative) or not. Format output like this: "Y" for yes and "N" for no.'

History:0 Context: Yes

In [530]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output11/output-11-0-Y.txt",history =0, context=True, max=40)

Skipped by GPT.

Convo:
 Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 
Skipped by GPT.

Convo:
 Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 
K Produced As an AI, I need the conversation above to analyze and answer your request, which you didn't provide. Could you please provide the conversation to identify whether the final message in it has a negative tone/intent or not? 
Prompt: The topic of the following conversation is Ordinary Life.
Person 1:  What about this ? State if the tone/intent of the last message in the conversation above is negative (only if it is extremely negative) or not. Format output like this: "Y" for yes and "N" for no.
Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Total Requests: 295


In [531]:
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output11/output-11-0-Y.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 264 264


**Stats**


Precision: 0.6666666666666666
Recall: 0.4583333333333333
F1 Score: 0.5432098765432098
Accuracy: 0.8593155893536122
FP 11
FN 26
TP 22
TN 204
Total: 263
Total Convos: 37


History: 1 Context: Yes

In [532]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output11/output-11-1-Y.txt",history =1, context=True, max=40)

Skipped by GPT.

Convo:
 Person 1:  And after that ? 
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 

Skipped by GPT.

Convo:
 Person 1:  What ? He can't do this to you . 
Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 

Skipped by GPT.

Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Person 2:  We'll see what happens after court . 

Total Requests: 294


In [534]:
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output11/output-11-1-Y.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 264 264


**Stats**


Precision: 0.6071428571428571
Recall: 0.3541666666666667
F1 Score: 0.4473684210526316
Accuracy: 0.8409090909090909
FP 11
FN 31
TP 17
TN 205
Total: 264
Total Convos: 37


History: 2 Context Yes

In [535]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output11/output-11-2-Y.txt",history =2, context=True, max=40)

Skipped by GPT.

Convo:
 Person 1: Mom , I want to get a motorcycle . Is that all right with you ? '
Person 2:  You mean a murder cycle ? Did you know more cyclists die in accidents than motorists ? '
Person 1:  Mom ! I'll wear a helmet and I'll drive sane . I promise . '
Person 2:  I'll tell you what . You can get a motorcycle on one condition . '
Person 1:  What's that ? '
Person 2:  You spend two weeks with dad in his ambulance as an EMT trainee and volunteer at the emergency room for one month . '
Person 1:  And after that ? '
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . '

Skipped by GPT.

Convo:
 Person 1: What's up ? You sound a little down in dumps . '
Person 2:  I quarreled with my roommate last night . He is really stubborn . '
Person 1:  Calm down ! Shouting won ’ t help ? '
Person 2:  He is really outrageous . '
Person 1:  What happened ? '
Person 2:  I went back home last night . You know tired as a dog , so I took a quick s

In [536]:
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output11/output-11-2-Y.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 274 274


**Stats**


Precision: 0.6571428571428571
Recall: 0.46
F1 Score: 0.5411764705882354
Accuracy: 0.8576642335766423
FP 12
FN 27
TP 23
TN 212
Total: 274
Total Convos: 38


History: 0 Context No

In [537]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output11/output-11-0-N.txt",history =0, context=False, max=40)

Skipped by GPT.

Convo:
 Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 
Skipped by GPT.

Convo:
 Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 
Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Total Requests: 295


In [539]:
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output11/output-11-0-N.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 264 264


**Stats**


Precision: 0.6666666666666666
Recall: 0.4166666666666667
F1 Score: 0.5128205128205129
Accuracy: 0.8560606060606061
FP 10
FN 28
TP 20
TN 206
Total: 264
Total Convos: 37


History: 1 Context No

In [540]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output11/output-11-1-N.txt",history =1, context=False, max=40)

Skipped by GPT.

Convo:
 Person 2:  What ? 
Person 1:  Any body piercing ? Stick out your tongue ! 

Skipped by GPT.

Convo:
 Person 1:  And after that ? 
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 

Skipped by GPT.

Convo:
 Person 1:  What ? He can't do this to you . 
Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 

Skipped by GPT.

Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Person 2:  We'll see what happens after court . 

Total Requests: 293


In [541]:
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output11/output-11-1-N.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 256 256


**Stats**


Precision: 0.6896551724137931
Recall: 0.425531914893617
F1 Score: 0.5263157894736842
Accuracy: 0.859375
FP 9
FN 27
TP 20
TN 200
Total: 256
Total Convos: 36


History: 2 Context No

In [542]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output11/output-11-2-N.txt",history =2, context=False, max=40)

Skipped by GPT.

Convo:
 Person 1: Mom , I want to get a motorcycle . Is that all right with you ? '
Person 2:  You mean a murder cycle ? Did you know more cyclists die in accidents than motorists ? '
Person 1:  Mom ! I'll wear a helmet and I'll drive sane . I promise . '
Person 2:  I'll tell you what . You can get a motorcycle on one condition . '
Person 1:  What's that ? '
Person 2:  You spend two weeks with dad in his ambulance as an EMT trainee and volunteer at the emergency room for one month . '
Person 1:  And after that ? '
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . '

Skipped by GPT.

Convo:
 Person 1: What's up ? You sound a little down in dumps . '
Person 2:  I quarreled with my roommate last night . He is really stubborn . '
Person 1:  Calm down ! Shouting won ’ t help ? '
Person 2:  He is really outrageous . '
Person 1:  What happened ? '
Person 2:  I went back home last night . You know tired as a dog , so I took a quick s

In [543]:
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output11/output-11-2-N.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 259 259


**Stats**


Precision: 0.6875
Recall: 0.4489795918367347
F1 Score: 0.5432098765432098
Accuracy: 0.8571428571428571
FP 10
FN 27
TP 22
TN 200
Total: 259
Total Convos: 37


Automated 

In [570]:
historyOptions=[0,1,2]
contextOptions=[True,False]
promptNo=["20"]
promptOptions=[
    'State if the tone/intent of the following message in the conversation above is negative (even if it is slightly negative) or not: \n @rukhshan@ \n Format output like this: "Y" for yes and "N" for no.'
    ]
groundPath = "/Users/rukhshanharoon/Desktop/LLMChattingApp/Analysis/angryReal40.txt"
maxCon=40
pathFinalOutput="/Users/rukhshanharoon/Desktop/LLMChattingApp/Analysis/finalOutput.csv"

if(len(promptNo)!=len(promptOptions)):
    raise ValueError("promptNo not equal to promptOptions!")

for promptIndex in range(len(promptOptions)):
    for historySelected in historyOptions:
        for contextSelected in contextOptions:
            folderName = "/Users/rukhshanharoon/Desktop/LLMChattingApp/Analysis/output"+promptNo[promptIndex]
            if not os.path.exists(folderName):
                os.makedirs(folderName)
            fileName = "output-"+promptNo[promptIndex]+"-"+str(historySelected)+"-"+ ('Y.txt' if contextSelected else 'N.txt')
            genPath = folderName + "/" + fileName
            labelsGPT = readSpecificConversations(textFile,angryConversations,promptOptions[promptIndex],genPath,historySelected, contextSelected, maxCon)
            angryOnlyRealLabels, angryOnlyGenLabels,n = processData(groundPath, genPath)
            if(len(angryOnlyRealLabels) != len(angryOnlyGenLabels)):
                print("Length of ground and gen not equal!")
                print(len(angryOnlyRealLabels), len(angryOnlyGenLabels))
                print("Issue in: ", fileName)
                df = PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n, promptNo[promptIndex],maxCon, promptOptions[promptIndex],historySelected,contextSelected, pathFinalOutput)
            else:
                

Final Prompt The topic of the following conversation is Ordinary Life.
Person 1: So Dick , how about getting some coffee for tonight ? State if the tone/intent of the following message in the conversation above is negative (even if it is slightly negative) or not: 
 Person 1: So Dick , how about getting some coffee for tonight ?  
 Format output like this: "Y" for yes and "N" for no.
Final Prompt The topic of the following conversation is Ordinary Life.
Person 2:  Coffee ? I don ’ t honestly like that kind of stuff . State if the tone/intent of the following message in the conversation above is negative (even if it is slightly negative) or not: 
 Person 2:  Coffee ? I don ’ t honestly like that kind of stuff .  
 Format output like this: "Y" for yes and "N" for no.
Final Prompt The topic of the following conversation is Ordinary Life.
Person 1:  Come on , you can at least try a little , besides your cigarette . State if the tone/intent of the following message in the conversation above

ZeroDivisionError: float division by zero

In [327]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompts[1],"outputSlightlyAngryGPT.txt", max=40)

Conversation index:  15
Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. END
Conversation index:  95
Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. END
Conversation index:  97
Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 5 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like t

In [353]:

angryOnlyRealLabels, angryOnlyGenLabels = processData('angryReal40.txt', 'outputSlightlyAngryGPT.txt')
print(len(angryOnlyRealLabels),len(angryOnlyGenLabels))

[17, 20, 25, 37]
17
20
25
37
40
251 251


In [354]:
print("Detects even slightly negative messages:") 
PRFA(angryOnlyRealLabels,angryOnlyGenLabels)

Detects even slightly negative messages:
Precision: 0.45121951219512196
Recall: 0.7708333333333334
F1 Score: 0.5692307692307692
Accuracy: 0.7768924302788844


In [335]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompts[2],"outputExtremelyAngryGPT.txt", max=40)


Conversation index:  15
Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. END
Conversation index:  95
Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. END
Conversation index:  97
Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like t

In [355]:
angryOnlyRealLabels, angryOnlyGenLabels = processData('angryReal40.txt', 'outputExtremelyAngryGPT.txt')
print(len(angryOnlyRealLabels),len(angryOnlyGenLabels))

[20, 25]
20
25
40
274 274


In [347]:
print("Detects extremely negative messages only:") 
PRFA(angryOnlyRealLabels,angryOnlyGenLabels)

Detects extremely negative messages only:
Precision: 0.627906976744186
Recall: 0.54
F1 Score: 0.5806451612903226
Accuracy: 0.8576642335766423


In [377]:
def printDialogue(index, filePath="/Users/rukhshanharoon/Desktop/LLMChattingApp/Analysis/Dataset/ijcnlp_dailydialog/dialogues_text.txt"):
    
    with open(filePath, 'r') as file:
        lines = file.readlines()
        if 0 <= index <= len(lines):
            convo = lines[index].split("__eou__")
            for utterance in convo:
                print(utterance)
        else:
            print("Line number is out of range.")


In [373]:
#conversations with at least one message labeled as angry
angryConversations = extractEmotionConversations(emotionFile,textFile,'1')


Conversation indexes with at least 1 angry emotion:
 {1, 2, 10251, 12301, 15, 32, 2084, 4132, 4134, 10281, 4143, 12337, 2100, 2101, 2102, 2106, 2111, 2113, 4161, 4165, 6214, 76, 2126, 4179, 4182, 8278, 8283, 2140, 4188, 95, 96, 97, 2148, 4196, 12395, 4205, 4208, 4211, 12403, 2168, 4216, 4217, 12408, 4220, 8318, 8321, 4227, 4228, 8324, 4231, 136, 12425, 4237, 4243, 10388, 10389, 10390, 10392, 10393, 155, 4251, 157, 10395, 10396, 4256, 4257, 4259, 4263, 6139, 2218, 10413, 12653, 2223, 2224, 4273, 178, 10415, 2228, 10416, 2230, 4278, 4280, 186, 4283, 2236, 2237, 10429, 2239, 2241, 2243, 4308, 2262, 4310, 4314, 222, 4320, 4321, 2275, 4327, 2280, 8428, 2285, 12530, 4348, 2302, 4354, 8452, 4362, 2324, 10517, 4377, 4379, 4387, 295, 296, 6440, 8487, 300, 4398, 306, 311, 8503, 4414, 10564, 2374, 327, 4423, 8521, 4426, 4427, 2380, 8522, 8526, 2383, 2384, 4431, 338, 2387, 8534, 2391, 351, 2402, 4451, 12642, 357, 364, 2413, 4460, 2415, 368, 2416, 370, 4465, 12284, 12652, 4470, 12656, 4472, 2427, 3

In [389]:
printDialogue(1)

So Dick , how about getting some coffee for tonight ? 
 Coffee ? I don ’ t honestly like that kind of stuff . 
 Come on , you can at least try a little , besides your cigarette . 
 What ’ s wrong with that ? Cigarette is the thing I go crazy for . 
 Not for me , Dick . 




In [393]:
printDialogue(95)
print("Conversation indexes with at least 1 angry emotion:\n",len(sorted(angryConversations)))

Jessie , I ’ m afraid I can ’ t come back home for dinner tonight . 
 Not back home for dinner again ? That ’ s the third time this week ! 
 I ’ m sorry . Our company has just opened . There are always too many things to handle . You know that . 
 You don ’ t have to explain . Suit yourself . 
 I apologize . You have my word , I ’ ll spend some time with you on the weekend . I promise . 
 We ’ ll see . 
 Thank you for understanding . I promise I ’ ll make it up to you . 


Conversation indexes with at least 1 angry emotion:
 508


In [409]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompts[3],"outputYesNoMaybeGPT.txt", max=40)


Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Skipped by GPT.

Convo:
 Person 1: Mark , can you dress the twins for me while I take a shower ? '
Person 2:  You don't know what you are asking ! '
Person 1:  Please ! You can do it . Their clothes are all laid out on the bed . '
Person 2:  Remember what happened last time ? '

Skipped by GPT.

Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Skipped by GPT.

Convo:
 Person 1: Mom , I want to get a motorcycle . Is that all right with you ? '
Person 2:  You mean a murder cycle ? Did you know more cyclists die in accidents than motorists ? '
Person 1:  Mom ! I'll wear a helmet and I'll drive sane . I promise . '
Person 2:  I'll tell you what . You can get a motorcycle on one condition . '
Person 1:  What's that ? '
Person 2:  You spend two weeks with dad in his ambulance as an EMT trainee and volunteer at the emergency room for one month . '
Person 1:  And after

In [426]:
angryOnlyRealLabels, angryOnlyGenLabels = processData('angryReal40.txt', 'outputYesNoMaybeGPT.txt')
print(len(angryOnlyRealLabels),len(angryOnlyGenLabels))

[17, 20, 25]
17
20
25
40
266 266


In [427]:
print("Detects extremely negative messages only:") 
PRFA(angryOnlyRealLabels,angryOnlyGenLabels)

Detects extremely negative messages only:
Precision: 0.8333333333333334
Recall: 0.46875
F1 Score: 0.6
Accuracy: 0.9074074074074074


In [416]:
printDialogue(sorted(angryConversations)[0])
print("Conversation indexes with at least 1 angry emotion:\n",len(sorted(angryConversations)))

So Dick , how about getting some coffee for tonight ? 
 Coffee ? I don ’ t honestly like that kind of stuff . 
 Come on , you can at least try a little , besides your cigarette . 
 What ’ s wrong with that ? Cigarette is the thing I go crazy for . 
 Not for me , Dick . 


Conversation indexes with at least 1 angry emotion:
 508


In [438]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompts[4],"outputYNMRecipientGPT.txt", max=40)


Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Skipped by GPT.

Convo:
 Person 1: Mark , can you dress the twins for me while I take a shower ? '
Person 2:  You don't know what you are asking ! '
Person 1:  Please ! You can do it . Their clothes are all laid out on the bed . '

Skipped by GPT.

Skipped by GPT.

Skipped by GPT.

Sleeping for 8.
Sleeping for 8.
Skipped by GPT.

Convo:
 Person 1: Mom , I want to get a motorcycle . Is that all right with you ? '
Person 2:  You mean a murder cycle ? Did you know more cyclists die in accidents than motorists ? '
Person 1:  Mom ! I'll wear a helmet and I'll drive sane . I promise . '
Person 2:  I'll tell you what . You can get a motorcycle on one condition . '

Sleeping for 8.
Skipped by GPT.

Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Skipped by GPT.

Convo:
 Person 1: What's up ? You sound a little down in dumps . '
Person 2:  I quarreled w

In [444]:
angryOnlyRealLabels, angryOnlyGenLabels = processData('angryReal40.txt', 'outputYNMRecipientGPT.txt')
print(len(angryOnlyRealLabels),len(angryOnlyGenLabels))

PRFA(angryOnlyRealLabels,angryOnlyGenLabels)

[17, 20, 25, 38]
17
20
25
38
40
261 261
Precision: 0.6829268292682927
Recall: 0.5957446808510638
F1 Score: 0.6363636363636364
Accuracy: 0.8773946360153256
Out of a total  41  messages flagged by GPT,  28  or  68.29268292682927 % were actually positive. Total number of positives in the ground truth is:  47 .


In [450]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompts[5],"outputYNRecipientGPT.txt", max=40)
angryOnlyRealLabels, angryOnlyGenLabels = processData('angryReal40.txt', 'outputYNRecipientGPT.txt')
print(len(angryOnlyRealLabels),len(angryOnlyGenLabels))
PRFA(angryOnlyRealLabels,angryOnlyGenLabels)

Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Skipped by GPT.

Convo:
 Person 1: Mark , can you dress the twins for me while I take a shower ? '
Person 2:  You don't know what you are asking ! '
Person 1:  Please ! You can do it . Their clothes are all laid out on the bed . '

Skipped by GPT.

Skipped by GPT.

Skipped by GPT.

Sleeping for 8.
Sleeping for 8.
Skipped by GPT.

Convo:
 Person 1: Mom , I want to get a motorcycle . Is that all right with you ? '
Person 2:  You mean a murder cycle ? Did you know more cyclists die in accidents than motorists ? '
Person 1:  Mom ! I'll wear a helmet and I'll drive sane . I promise . '
Person 2:  I'll tell you what . You can get a motorcycle on one condition . '

Sleeping for 8.
Skipped by GPT.

Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Sleeping for 8.
Skipped by GPT.

Convo:
 Person 1: What's up ? You sound a little down in dumps . '
Person 2:  I quarreled w